# Network Dynamics and Learning: Homework I
## Alessandro Casella (S306081)
## s306081@studenti.polito.it
### Politecnico di Torino

In [1]:
# https://networkx.org/
import networkx as nx

# https://docs.scipy.org/doc/scipy/reference/misc.html
import scipy.misc
# https://docs.scipy.org/doc/scipy/tutorial/io.html
from scipy import io

import numpy as np

# https://www.cvxpy.org/
import cvxpy as cp